## This notebook is used to test and develop the integration of Kamodo into GEODYN


- We make a some critical modifications to GEODYN for the purpose of quick testing:
    1. We use a much lager `STEP` size to go through the runs faster.  
       `15.00 (m)  ->  60.00 (m)`
    


## Run flow of Pygeodyn to call fortran which subsequently calls python

GEODYN is split into two programs: 1) `IIS` which is a scheduler to organize files and inputs, and 2) `IIE` which is an execution of the orbit determination program.


From Jupyter Notebook:  
  1. Initialize an Pygeodyn Object and tell it to run a prepared geodyn simulation 
      ```
      Obj_Geodyn = Pygeodyn(run_params1)
      Obj_Geodyn.RUN_GEODYN()
      ```
      1. `RUN_GEODYN()` calls functions in the `PygeodynController()` class to organizes the inputs in python and prepare them in a temporary directory.
         1.  From a temporary directory python sends a command to the command line to execute the already compiled fortran executable.
          - run IIS first:
          ```
          command_IIS = self.G2SDIR+'/giis2002_gfortran > '+'iisout 2> '+'iiserr'
          subprocess.run(command_IIS, shell = True)
          ```
          - run IIE after IIS:
          ```
          command_IIE = self.G2EDIR+'/giie2002_gfortran > '+'iieout 2> '+'iieerr'
          subprocess.run(command_IIE, shell = True)
          ```
            1. Within the `IIE` fortran executable, the entire orbit propagation takes place. Dynamic arrays are built and used to store changing variables from each subroutine.  Seemingly mysterious inputs and outputs are passed into and out of each subroutine this way.
                1.  In the `DRAG.F90` a density model is indexed at every timestep of the orbit.  This doesn't happen all at once-- a single timestep is given to drag which grabs a density from a model calcualtes the drag acceleration at that timestep and returns the drag acceleration to the to the higher levels of subroutines for further calculating the Variational and normal equations in sequence with the other accelerations at that timestep.  These values are used to propagate the satellite forward one timestep and then repeat the process. All of this is to say that in practice, we must index the density models with a single ephemeris point instead of with an array of values.
                    1. `DRAG.f90` calls `KamodoModels.f90` which uses `EXECUTE_COMMAND_LINE("python SingleSatelliteFlythrough TIEGCM lon lat alt (etc.) ")` to call the single point of density from Kamodo-TIEGCM Reader.


**IN SUMMARY**:
- From python send `subprocess.run()` command to execute `Fortran` executable. (Python is now waiting in the background)
  - From `Fortran` run a shell command with `EXECUTE_COMMAND_LINE()` to run the `python script`
         


In [1]:
%load_ext autoreload
%autoreload 2
import copy
import time
import sys  


sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
from PYGEODYN import Pygeodyn


### Identify which arcs you want to run:
arcs_files = [ 
            '2018.336',  # 1
             ]

#------ A dictionary containing the run parameters ------  
run_params = {} 
run_params['arc']              =   arcs_files
run_params['satellite']        =  'icesat2'  
run_params['den_model']        =  'tiegcm_cl'  
run_params['SpecialRun_name']  =  'CL_kamodo_interface'  
run_params['verbose']          =  False


%load_ext autoreload
%autoreload 2

import pickle 
sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
from PYGEODYN import Pygeodyn

for imodel,val_model in enumerate( ['tiegcm_cl']):
    run_params1 = copy.deepcopy(run_params)
    run_params1['den_model']  =  val_model  
    run_params1['action']           =  'run'

    ### Load the data into an object
    Obj_Geodyn = Pygeodyn(run_params1)
    Obj_Geodyn.RUN_GEODYN()
    del Obj_Geodyn
    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
                      ......... RUNNING GEODYN
Run # 1     Current Time =      15:05:28
Run # 1
Run # 1     Cleaning iisset... :    /data/data_geodyn/inputs/icesat2/setups/iisset.2018.336.bz2
Epoch Start:  2018-12-01 21:00:00
+ ——————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
|
| ---------------------- Some run information ----------------------
|
|  Run # 1     IISSET Cleaned      tmp/.../cleaned_setup_2018336
|  Run # 1     Density Model:      tiegcm_cl
|  Run # 1     GEODYN Version:     Kamodo_pygeodyn_MODS
|  Run # 1     ARC run:            icesat2_2018336_54hr.tiegcm_cl
|  Run # 1     Output directory:   /data/data_geodyn/results/icesat2/tiegcm_cl/tiegcm_cl_acceloffCL_kamodo_interface
|  Run # 1     EXAT File:          /data/data_geodyn/inputs/icesat2/external_attitude/EXAT01.2018.336.gz
|
+ ——————————————————————————————————

In [2]:
%load_ext autoreload
%autoreload 2

# import pickle 
# sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
# from PYGEODYN import Pygeodyn

for imodel,val_model in enumerate( ['tiegcm']):
    read_params = copy.deepcopy(run_params)
    read_params['den_model']  =  val_model  
    read_params['action']  =  'read'  
    read_params['request_data']      = ['AdjustedParams',
                                        'Trajectory_orbfil',
                                        'Density',
                                        'Residuals_obs',
                                        'Residuals_summary',
                                        ]

    ### Load the data into an object
    Obj_Geodyn = Pygeodyn(read_params)
    Obj_Geodyn.getData()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Density model string formats: [msis86, msis00, msis2, dtm87, jaachia71]
Dear zach, please add the remaining models you have added. --love zach
                      ......... READING GEODYN output


AttributeError: 'Pygeodyn' object has no attribute 'DEN_DIR'